In [1]:
# Webscraping Application
# Importing Dependencies

from flask import Flask, render_template, redirect
from pandas import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import time

In [2]:
# Application Name
mars_site = Flask(__name__)

# Defining procedures to scrape a number of websites

executable_path = {"executable_path": "C:/webdrivers/chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
url = ["https://redplanetscience.com/", "https://spaceimages-mars.com/image/featured/mars3.jpg", "https://galaxyfacts-mars.com","https://marshemispheres.com/"]


In [3]:
# Scraping News From the Mars website using Splinter 
browser.visit(url[0])
html = browser.html
soup = BeautifulSoup(html, "html.parser")
    
mars_info = {}
    
mars_info['latest_title'] = soup.find("div", class_ = "content_title").get_text()
mars_info['latest_paragraph'] = soup.find("div", class_ = "article_teaser_body").get_text()
    
#browser.quit()

In [4]:
print(mars_info)

{'latest_title': "NASA's Mars Perseverance Rover Gets Its Sample Handling System", 'latest_paragraph': 'The system will be collecting and storing Martian rock and soil. Its installation marks another milestone in the march toward the July launch period.'}


In [8]:
# Visit JPL Featured Space Image url
url_spaceimage = url[1]
browser.visit(url_spaceimage)

# HTML Object
img_html = browser.html
img_soup = BeautifulSoup(img_html, "html.parser")

# Display url
main_url = url[1]

mars_info["featured_image_url"] = main_url

In [10]:
# Visit the Mars Facts webpage and use Pandas to scrape the table
url_facts = url[2]

# Use Pandas - read_html - to scrape tabular data from a page
mars_facts = pd.read_html(url_facts)

# Find the mars facts DataFrame in the list
mars_df = mars_facts[0]

# Create Data Frame
mars_df.columns = ["Description", "Value", " "]

# Set index to Description
mars_df.set_index("Description", inplace=True)

# Save html code to folder Assets
html_table = mars_df.to_html()

# Strip unwanted newlines to clean up the table
html_table.replace("\n", '')

# Save html code
mars_df.to_html("mars_facts_data.html")

mars_info["mars_facts"] = html_table

In [ ]:
# Visit the Mars Hemisphere website link
browser.visit(url[3])

# HTML Object
html_hemisphere = browser.html
hem_soup = BeautifulSoup(html_hemisphere, "html.parser")

# Find containers whcih has mars hemispheres information
hemispheres = hem_soup.find_all("div", class_="item")

# Create empty list
hemispheres_info = []

# Sign main url for loop
hemispheres_url = url[3]

# Loop through the list of all hemispheres information
for i in hemispheres:
    title = i.find("h3").text
    hemispheres_img = i.find("a", class_="itemLink product-item")["href"]

    # Visit the link that contains the full image website 
    browser.visit(hemispheres_url + hemispheres_img)

    # HTML Object
    image_html = browser.html
    web_info = BeautifulSoup(image_html, "html.parser")

    # Create full image url
    img_url = hemispheres_url + web_info.find("img", class_="wide-image")["src"]

    mars_info["title"] = title.strip()       
    mars_info["img_url"] = img_url

    hemispheres_info.append({"title" : title, "img_url" : img_url})

    mars_info["hemispheres_info"] = hemispheres_info

In [ ]:
#Quits browser after scrape procedure
browser.quit()